## mz-rt matching from Authentic Compounds to Feature Table 

- Refcatoring old pieces of code 
- May not be the optimal solution but I was in hurry 


--- 
Notes for myself: 
- Dont forget the correct order of targets list or else change in function; Which is smpd name , mz rt and everything else e.g 'HMDB0042008', 220.1332, 88.95, ['Ritalinic acid']
- no need of sorting FT;Taken care through indexing tree; 
- However - Use a subset version with three columns in following order ; mz	rtime	id_number for FT 

AS; August-09-2022

In [1]:
import json 
import pandas as pd

In [2]:
#read json file of authentic stanfdard library prep
auth_file="./json_MG_JMS/combn_1strnd_HILICpos_2022-04-19_Shujian Zheng|Minghao Gong.json"
list_compounds = json.load(open(auth_file))
my_auth_list=list(list_compounds.values())[: 1][0]
# first element is "list_of_Empirical_Compounds": therefore [: 1][0]th item

In [3]:
#my_auth_list[:2]

## Functions 
- for Converting AuthenticLIbrary to CSV 
- Matching mz lists 

In [4]:
def ConvertAuthLibrarytoCsv(my_auth_list): 
    mz =[]
    rt= []
    ion_relation= []
    compounds=[]
    names=[]
    #iterate for every item (empcmpd) of dictonary 
    for i in my_auth_list:
        #print(i)
        mydb_i=i#
        #list of features ; acquire mz and rt for each feature 
        mydb_i_list_features=mydb_i['list_features']
        mz+=[mydb_i_list_features['mz'] for mydb_i_list_features in mydb_i_list_features if 'mz' in mydb_i_list_features]
        rt+=[mydb_i_list_features['rtime'] for mydb_i_list_features in mydb_i_list_features if 'rtime' in mydb_i_list_features]
        #list of compounds ; acquire compound name and db id 
        mydb_j_identity=mydb_i['identity'] 

        if len([mydb_i_list_features['mz'] for mydb_i_list_features in mydb_i_list_features if 'mz' in mydb_i_list_features]) > 0:
            compounds+=[d['compounds'] for d in mydb_j_identity if 'compounds' in d]*len([mydb_i_list_features['mz'] for mydb_i_list_features in mydb_i_list_features if 'mz' in mydb_i_list_features])
            names+=[d['names'] for d in mydb_j_identity if 'names' in d]*len([mydb_i_list_features['mz'] for mydb_i_list_features in mydb_i_list_features if 'mz' in mydb_i_list_features])
    df = pd.DataFrame(list(zip(mz, rt,  compounds, names)), columns=['mz', "rt", "compounds", "names"])
    return (df)


In [5]:
def find_targets(targetList, featuresTree, result_file):
    # target format: name at col 0, m/z at col 1 and rt col 2
    # feature format: row_number, m/z, rt, original line
    matched = []
    # filter targetList
    new = []
    min_mz, max_mz = min(featuresTree.keys()), max(featuresTree.keys())
    for x in targetList:
        if min_mz < x[1] < max_mz:
            new.append(x)
        else:
            print("out of m/z range: ", x)
    for x in new:
        # mz -0.1 to + 0.1 as search range
        neighbors = set([int(x[1]-0.1), int(x[1]+0.1)])
        for ii in neighbors:
            for F in featuresTree[ii]:
                if match2( (x[1], x[2]), F[1:3] ):
                    matched.append((x + F))
                
    if matched:
        s = "#Matched result\n"
        for line in matched:
            s += '\t'.join([str(x) for x in line]) + '\n'
        with open(result_file, 'w') as file:
            file.write(s)
        print("Found %d matches, result written to %s" %(len(matched), result_file))
    else:
        print("No match found")
        
    return matched

## Step1;Convert Authentic Compounds Json file to csv 

In [6]:
AuthCmpdData=ConvertAuthLibrarytoCsv(my_auth_list)
AuthCmpdData['rt'] = [round(x,2) for x in AuthCmpdData['rt']]
AuthCmpdData['mz'] = [round(x,4) for x in AuthCmpdData['mz']]

In [7]:
AuthCmpdData

,mz,rt,compounds,names
0,220.1332,88.95,HMDB0042008,[Ritalinic acid]
1,120.0655,163.95,HMDB0000719,[L-Homoserine]
2,190.1186,171.51,HMDB0000679,[Homocitrulline]
3,106.0499,172.63,HMDB0000187,[L-Serine]
4,120.0655,161.51,HMDB0000167,[L-Threonine]
...,...,...,...,...
115,389.1627,21.60,HMDB0005032,Cetirizine
116,389.1627,50.40,HMDB0005032,Cetirizine
117,338.0522,21.60,HMDB0001919,Famotidine
118,338.0522,39.60,HMDB0001919,Famotidine


In [8]:
#sometimes i need to work in r ; export for that 
AuthCmpdData.to_csv("Combination_HILICPOS_AS.csv", sep=',',index=False) 
AuthCmpdData.shape

(120, 4)

In [9]:
# I need compound name first , then mz and then rt as my atarget list; this is how its coded in my mz-rt match function the target list 
AuthCmpdData = AuthCmpdData[["compounds", "mz","rt","names"]]

In [10]:
AuthCmpdData=AuthCmpdData.values.tolist()

In [11]:
AuthCmpdData[:3]

[['HMDB0042008', 220.1332, 88.95, ['Ritalinic acid']],
 ['HMDB0000719', 120.0655, 163.95, ['L-Homoserine']],
 ['HMDB0000679', 190.1186, 171.51, ['Homocitrulline']]]

## Step2: Match with feature table  based on mz-rt

In [12]:
# i am using a subset version with three columns in following order ; 
#mz	rtime	id_number
feature_table = "HP_preferred_Feature_table_AS.tsv"

## Functions
# m/z at col 1, rt at col 2

row = 0
features = []
for line in open(feature_table).readlines()[1:]:
    a = line.split('\t')
    row += 1
    features.append([ 'row'+str(row), float(a[0]), float(a[1]), line ])
    
print (features[:3])



[['row1', 80.7236, 239.58, '80.7236\t239.58\tF2\n'], ['row2', 80.9482, 137.79, '80.9482\t137.79\tF3\n'], ['row3', 80.9482, 157.75, '80.9482\t157.75\tF4\n']]


In [13]:
tree = {}
all_mzs = [x[1] for x in features]
min_mz, max_mz = min(all_mzs), max(all_mzs)
print("min_mz, max_mz = ", min_mz, max_mz)

for ii in range(int(min_mz), int(max_mz)+1):
    tree[ii] = []

# building feature index so that the search isn't too slow.
for f in features:
    tree[int(f[1])].append(f)

min_mz, max_mz =  80.7236 999.657


In [14]:
# 0.000010 is 10 ppm
PPM_tolerance = 0.000010
# use a large number to include anything in RTime, small number to be specific
RTime_tolerance = 10      
             
# F1 = (m/z, rt)
def match2(F1, F2):
    if abs(F1[0]-F2[0])/F1[0] < PPM_tolerance and abs(F1[1] - F2[1]) < RTime_tolerance:
        return True
    else:
        return False
    
# test
print(match2((129.1541, 55), (129.1533, 144)))
print(match2((129.1541, 55), (129.1533, 61)))

False
True


In [15]:
result_file = "search_result.tsv"

matched = find_targets( AuthCmpdData, tree, result_file )

Found 30 matches, result written to search_result.tsv
